------------------------------- /!\ NE PLUS TOUCHER EN BAS /!\ --------------------------------------

In [2]:
df_animal = pd.read_csv("C:/Users/namor/Dropbox/Simplon/Programmation/Projet Fil rouge/Data/2018/FAOSTAT Produit Animal.csv")
df_animal = df_animal.drop(columns=['Item', "Domain Code", "Domain", "Element Code", "Year Code", "Flag", "Flag Description"])

df_vegetal = pd.read_csv("C:/Users/namor/Dropbox/Simplon/Programmation/Projet Fil rouge/Data/2018/FAOSTAT Produit Vegetal.csv")
df_vegetal = df_vegetal.drop(columns=['Item', "Domain Code", "Domain", "Element Code", "Year Code", "Flag", "Flag Description"])

df_cereale = pd.read_csv("C:/Users/namor/Dropbox/Simplon/Programmation/Projet Fil rouge/Data/2018/FAOSTAT Produit Céréale par pays.csv")
df_cereale = df_cereale.drop(columns=['Item', "Domain Code", "Domain", "Element Code", "Year Code", "Flag", "Flag Description"])

df_details_produit = pd.concat([df_animal, df_vegetal, df_cereale]).drop_duplicates().reset_index(drop=True)
df_details_produit.insert(loc=0, column='Area_1', value=df_details_produit['Area'])
df_details_produit = df_details_produit.set_index('Area_1')
df_details_produit[["Area"]] = df_details_produit[["Area"]].replace({'Belgium-Luxembourg': 'Luxembourg', 'Ethiopia PDR': 'Ethiopia', 'Netherlands Antilles (former)': 'Netherlands', 'Serbia and Montenegro': 'Montenegro', 'Sudan (former)': 'Sudan', 'Sudan (former)': 'Sudan', 'Sudan (former)': 'Sudan'})

# Enregistrer le DataFrame dans un fichier csv
# df_details_produit.to_csv('CSV/Bilan Alimentaire.csv', index=False, encoding="utf-8-sig", quoting=csv.QUOTE_NONNUMERIC)
# df_details_produit.to_csv('CSV/Bilan Alimentaire.csv', index=False, sep=',', encoding='latin1')
# df_details_produit.to_csv('CSV/Bilan Alimentaire.csv', index=False, sep=',', encoding="utf-8-sig")

df_details_produit = df_details_produit.fillna(0)

dictionnaire = {}
for k in df_details_produit.Area.unique():
    dictionnaire[k] = {}

    for l in df_details_produit[df_details_produit["Area"] == k]["Item Code"].unique():
        dictionnaire[k][l] = {}
    
        for i in df_details_produit[df_details_produit["Area"] == k]["Element"].unique():
            liste = []
            for j in df_details_produit[(df_details_produit["Area"] == k) & (df_details_produit["Item Code"] == l) & (df_details_produit["Element"] == i)]["Value"]:
                liste.append(j)
                dictionnaire[k][l][i] = liste
                
df = pd.concat({k: pd.DataFrame(v).T for k, v in dictionnaire.items()}, axis=0)

bf = df.copy()
bf.replace(np.NaN, '[0.0]', inplace=True)

for a in bf:
    bf[a] = bf[a].apply(lambda x: str(x).strip('[]').split(', '))
    bf[a] = bf[a].apply(lambda x: float(str(x[0]).strip('[]')))

bf = bf.reset_index()
bf[["level_0"]] = bf[["level_0"]].replace({'Belgium-Luxembourg': 'Luxembourg', 'Ethiopia PDR': 'Ethiopia', 'Netherlands Antilles (former)': 'Netherlands', 'Serbia and Montenegro': 'Montenegro', 'Sudan (former)': 'Sudan', 'Sudan (former)': 'Sudan', 'Sudan (former)': 'Sudan'})
bf.insert(loc=0, column='level_0_1', value=bf['level_0'])
bf = bf.set_index('level_0_1')

bf_pays = pd.read_csv("C:/Users/namor/Dropbox/Simplon/Programmation/Projet Fil rouge/Version 2/CSV/Pays.csv")
bf_pays.insert(loc=0, column='Area_1', value=bf_pays['Area'])
bf_pays = bf_pays.set_index('Area_1')

bf = pd.merge(bf, bf_pays['Area Code'], how='left', left_index=True, right_index=True)
bf[['Area Code']] = bf[['Area Code']].astype(np.int64)
bf = bf.rename(columns = {'level_0': 'Area', 'level_1': 'Item Code'})
bf = bf.reset_index(drop=True)
# bf = bf[["Area Code", "Item Code", "Production", "Import Quantity", "Stock Variation", "Domestic supply quantity", "Processing", "Residuals", "Food", "Food supply quantity (kg/capita/yr)", "Food supply (kcal/capita/day)", "Protein supply quantity (g/capita/day)", "Fat supply quantity (g/capita/day)", "Export Quantity", "Losses", "Feed", "Seed", "Other uses (non-food)", "Tourist consumption"]]
cols = bf.columns.tolist()
cols = cols[-1:] + cols[:-1]
bf = bf[cols]

# bf = bf.fillna([0.0])

liste = []

for i in range(16349):
    liste.append(2018)
    
bf.insert(1, column= "Year", value=liste)
# bf = bf.drop_duplicates(subset=['Area Code', 'Item Code'], keep='last')

# Enregistrer le DataFrame dans un fichier csv
bf.to_csv('CSV/Bilan Alimentaire.csv', index=False, sep=',', encoding="utf-8-sig")

In [3]:
bf

,Area Code,Year,Area,Item Code,Production,Import Quantity,Stock Variation,Domestic supply quantity,Processing,Residuals,...,Food supply quantity (kg/capita/yr),Food supply (kcal/capita/day),Protein supply quantity (g/capita/day),Fat supply quantity (g/capita/day),Export Quantity,Losses,Feed,Seed,Other uses (non-food),Tourist consumption
0,2,2018,Afghanistan,2731,130.0,5.0,-1.0,135.0,0.0,0.0,...,3.63,21.0,1.51,1.66,0.0,0.0,0.0,0.0,0.0,0.0
1,2,2018,Afghanistan,2732,161.0,0.0,0.0,160.0,0.0,0.0,...,4.31,27.0,1.68,2.24,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2018,Afghanistan,2733,0.0,0.0,0.0,0.0,0.0,0.0,...,0.01,0.0,0.01,0.02,0.0,0.0,0.0,0.0,0.0,0.0
3,2,2018,Afghanistan,2734,29.0,25.0,0.0,53.0,0.0,0.0,...,1.43,5.0,0.51,0.31,0.0,0.0,0.0,0.0,0.0,0.0
4,2,2018,Afghanistan,2735,12.0,0.0,0.0,12.0,0.0,0.0,...,0.31,1.0,0.14,0.05,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16344,181,2018,Zimbabwe,2657,145.0,1.0,-15.0,161.0,0.0,0.0,...,11.17,10.0,0.15,0.00,0.0,0.0,0.0,0.0,0.0,0.0
16345,181,2018,Zimbabwe,2658,64.0,1.0,0.0,65.0,0.0,0.0,...,4.51,36.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0
16346,181,2018,Zimbabwe,2659,9.0,0.0,0.0,9.0,0.0,0.0,...,0.00,0.0,0.00,0.00,1.0,0.0,0.0,0.0,9.0,0.0
16347,181,2018,Zimbabwe,2680,0.0,0.0,0.0,0.0,0.0,0.0,...,0.01,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0


------------------------------- /!\ NE PLUS TOUCHER EN HAUT /!\ --------------------------------------

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns
import ipywidgets as widgets
import csv

In [6]:
df_2 = pd.read_csv("C:/Users/namor/Dropbox/Simplon/Programmation/Projet Fil rouge/Version 2/CSV/Bilan Alimentaire.csv")
del df_2['Area']

# Enregistrer le DataFrame dans un fichier csv
df_2.to_csv('CSV/Bilan Alimentaire_2.csv', index=False, sep=',', encoding="utf-8-sig")
df_2

,Area Code,Year,Item Code,Production,Import Quantity,Stock Variation,Domestic supply quantity,Processing,Residuals,Food,Food supply quantity (kg/capita/yr),Food supply (kcal/capita/day),Protein supply quantity (g/capita/day),Fat supply quantity (g/capita/day),Export Quantity,Losses,Feed,Seed,Other uses (non-food),Tourist consumption
0,2,2018,2731,130.0,5.0,-1.0,135.0,0.0,0.0,135.0,3.63,21.0,1.51,1.66,0.0,0.0,0.0,0.0,0.0,0.0
1,2,2018,2732,161.0,0.0,0.0,160.0,0.0,0.0,160.0,4.31,27.0,1.68,2.24,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2018,2733,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.01,0.02,0.0,0.0,0.0,0.0,0.0,0.0
3,2,2018,2734,29.0,25.0,0.0,53.0,0.0,0.0,53.0,1.43,5.0,0.51,0.31,0.0,0.0,0.0,0.0,0.0,0.0
4,2,2018,2735,12.0,0.0,0.0,12.0,0.0,0.0,12.0,0.31,1.0,0.14,0.05,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16344,181,2018,2657,145.0,1.0,-15.0,161.0,0.0,0.0,161.0,11.17,10.0,0.15,0.00,0.0,0.0,0.0,0.0,0.0,0.0
16345,181,2018,2658,64.0,1.0,0.0,65.0,0.0,0.0,65.0,4.51,36.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0
16346,181,2018,2659,9.0,0.0,0.0,9.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,1.0,0.0,0.0,0.0,9.0,0.0
16347,181,2018,2680,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0


In [2]:
df_2 = pd.read_csv("C:/Users/namor/Dropbox/Simplon/Programmation/Projet Fil rouge/Version 2/CSV/Bilan Alimentaire_2.csv")
df_2

,Area Code,Year,Item Code,Production,Import Quantity,Stock Variation,Domestic supply quantity,Processing,Residuals,Food,Food supply quantity (kg/capita/yr),Food supply (kcal/capita/day),Protein supply quantity (g/capita/day),Fat supply quantity (g/capita/day),Export Quantity,Losses,Feed,Seed,Other uses (non-food),Tourist consumption
0,2,2018,2731,130.0,5.0,-1.0,135.0,0.0,0.0,135.0,3.63,21.0,1.51,1.66,0.0,0.0,0.0,0.0,0.0,0.0
1,2,2018,2732,161.0,0.0,0.0,160.0,0.0,0.0,160.0,4.31,27.0,1.68,2.24,0.0,0.0,0.0,0.0,0.0,0.0
2,2,2018,2733,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.01,0.02,0.0,0.0,0.0,0.0,0.0,0.0
3,2,2018,2734,29.0,25.0,0.0,53.0,0.0,0.0,53.0,1.43,5.0,0.51,0.31,0.0,0.0,0.0,0.0,0.0,0.0
4,2,2018,2735,12.0,0.0,0.0,12.0,0.0,0.0,12.0,0.31,1.0,0.14,0.05,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
16344,181,2018,2657,145.0,1.0,-15.0,161.0,0.0,0.0,161.0,11.17,10.0,0.15,0.00,0.0,0.0,0.0,0.0,0.0,0.0
16345,181,2018,2658,64.0,1.0,0.0,65.0,0.0,0.0,65.0,4.51,36.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0
16346,181,2018,2659,9.0,0.0,0.0,9.0,0.0,0.0,0.0,0.00,0.0,0.00,0.00,1.0,0.0,0.0,0.0,9.0,0.0
16347,181,2018,2680,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.01,0.0,0.00,0.00,0.0,0.0,0.0,0.0,0.0,0.0


------------------------------------------------------------

In [3]:
df_valeur = pd.read_csv("C:/Users/namor/Dropbox/Simplon/Programmation/Projet Fil rouge/Data/2018/FAOSTAT Produit Animal.csv")
for i in df_valeur[df_valeur["Unit"] == "1000 tonnes"]["Element"].unique():
    print(i)
# for i in df_valeur[df_valeur["Unit"] == "kg"]["Element"].unique():
#     print(i)
# for i in df_valeur[df_valeur["Unit"] == "kcal/capita/day"]["Element"].unique():
#     print(i)
# for i in df_valeur[df_valeur["Unit"] == "g/capita/day"]["Element"].unique():
#     print(i)

True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
False
False
False
False
True
True
True
True
True
Tr

False
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
False
False
False
False
True
True


True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
False
False
False
False
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
True
True
False
False
False
False
Tru

False
True
True
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True


True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True


...

True
False
False
False
False
True
True
True
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
False
False


True
True
False
False
False
False
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
T

True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
False
False
False
False
True
True
True
True
False
False
False
False
True
True
True
True
False
False
False
False
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
False
False


True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
False
False
Fals

False
False
False
False
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
False
False
False
False
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
True
False
False
False
False
True
True
True
True
True
True
False
False
False

In [17]:
df_valeur['Unit'].unique()

array(['1000 tonnes', 'kg', 'kcal/capita/day', 'g/capita/day'],
      dtype=object)

In [5]:
bf[bf["Area Code"] == 424]

,Area Code,Year,Area,Item Code,Production,Import Quantity,Stock Variation,Domestic supply quantity,Processing,Residuals,...,Food supply quantity (kg/capita/yr),Food supply (kcal/capita/day),Protein supply quantity (g/capita/day),Fat supply quantity (g/capita/day),Export Quantity,Losses,Feed,Seed,Other uses (non-food),Tourist consumption
16148,424,2018,Yugoslav SFR,2781,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16149,424,2018,Yugoslav SFR,2782,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16150,424,2018,Yugoslav SFR,2761,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16151,424,2018,Yugoslav SFR,2762,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16152,424,2018,Yugoslav SFR,2763,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16153,424,2018,Yugoslav SFR,2764,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16154,424,2018,Yugoslav SFR,2765,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16155,424,2018,Yugoslav SFR,2766,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
16156,424,2018,Yugoslav SFR,2767,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
